In [ ]:
# download train_sample
from huggingface_hub import snapshot_download, login
import os

login('')

os.makedirs("PhysicalAI-Spatial-Intelligence-Warehouse", exist_ok=True)

snapshot_download(
    repo_id="nvidia/PhysicalAI-Spatial-Intelligence-Warehouse",
    repo_type="dataset",
    local_dir="PhysicalAI-Spatial-Intelligence-Warehouse",
    allow_patterns="train_sample/*"
)

/root/miniconda3/envs/spatialrgpt_finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 201 files: 100%|██████████| 201/201 [00:03<00:00, 50.84it/s]


'/root/SpatialRGPT/PhysicalAI-Spatial-Intelligence-Warehouse'

In [2]:
# move train_sample.json to like data folder stuture
!cp PhysicalAI-Spatial-Intelligence-Warehouse/train_sample/train_sample.json PhysicalAI-Spatial-Intelligence-Warehouse

In [1]:
# download pretrain mdoel
!git lfs install
!huggingface-cli download a8cheng/SpatialRGPT-VILA1.5-8B \
    --local-dir checkpoints/SpatialRGPT-VILA1.5-8B \
    --local-dir-use-symlinks False

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log


/root/miniconda3/envs/spatialrgpt_finetune/lib/python3.10/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 20 files: 100%|██████████████████████| 20/20 [00:00<00:00, 2021.06it/s]
/root/SpatialRGPT/checkpoints/SpatialRGPT-VILA1.5-8B


In [2]:
# Load model from HuggingFace or Dowloaded Checkpoint
from llava.mm_utils import get_model_name_from_path
from llava.model.builder import load_pretrained_model # To get tokenizer and image_processor

args_model_path = "checkpoints/SpatialRGPT-VILA1.5-8B"
# args_model_path = "checkpoints/SpatialRGPT-VILA1.5-8B"
args_model_base = None
args_conv_mode = "llava_v1"

model_name_inferred = get_model_name_from_path(args_model_path)
print(f"model_name_inferred: {model_name_inferred}")

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path = args_model_path,
    model_name = model_name_inferred,
    model_base = args_model_base,
    load_8bit=False,
    load_4bit=False,
    device_map='auto',
    device='cuda'
)
print("SpatialRGPT model loaded Successfully!")

/root/miniconda3/envs/spatialrgpt_finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


[2025-06-04 10:04:41,827] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
model_name_inferred: SpatialRGPT-VILA1.5-8B


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.05s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Resuming region extractor from:  checkpoints/SpatialRGPT-VILA1.5-8B/region_extractor
SpatialRGPT model loaded Successfully!


In [1]:
# download file of dataset from huggingface
from huggingface_hub import hf_hub_download

hf_hub_download(
    repo_id="nvidia/PhysicalAI-Spatial-Intelligence-Warehouse",
    repo_type="dataset",
    filename="train.json",
    local_dir="PhysicalAI-Spatial-Intelligence-Warehouse"
)

'PhysicalAI-Spatial-Intelligence-Warehouse/train.json'

In [2]:
# Read 
# Read train.json
import json
filename = "PhysicalAI-Spatial-Intelligence-Warehouse/train.json"
with open(filename, 'r') as f:
    train_data = json.load(f)
print(f"Number of samples: {len(train_data)}")

Number of samples: 499083


In [3]:
train_data[0].keys()

dict_keys(['id', 'image', 'conversations', 'rle', 'category', 'normalized_answer', 'freeform_answer'])

In [39]:
import random

idx = random.randint(0, len(train_data))
len(train_data[idx]['conversations'])

2

In [69]:
def is_single_turn_conversation(samples):
    num_turns = [len(sample['conversations'])/2 for sample in samples]
    return sum(num_turns) == float(len(samples))

samples = random.sample(train_data, 1000)
is_single_turn_conversation(samples)

True

In [72]:
train_data[0]['image'].split('.')

['070760', 'png']

In [88]:
def is_all_png(samples):
    all_png = [1 for sample in samples if sample['image'].split('.')[-1] == 'png']
    return sum(all_png) == len(samples)
samples = [
    {'image': "001.png"},
    {'image': "001.jpg"}
]
# samples = random.sample(train_data, 1000)
is_all_png(samples)

False

In [97]:
samples = random.sample(train_data, 1000)
is_all_png(samples)

True

In [7]:
from PIL import Image
import numpy as np

depth = Image.open("PhysicalAI-Spatial-Intelligence-Warehouse/train_sample/depths/001511_depth.png")
depth.mode, np.asarray(depth).shape

('L', (1080, 1920))

In [10]:
if depth.mode != 'RGB':
    depth_rgb = depth.convert('RGB')
depth_rgb.mode, np.asarray(depth_rgb).shape

('RGB', (1080, 1920, 3))

In [12]:
depth_rgb.mode != 'RGB'

False